# Path Divergence Analysis

This notebook analyzes why FlexDTW and Parflex paths intersect at a point and then diverge by examining the cost matrix and path decisions around that intersection.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import import_ipynb

# Import required modules
import Parflex
import FlexDTW

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 1. Data Loading & Setup

In [2]:
# File paths
file_1 = "/home/ijain/ttmp/Chopin_Mazurkas_features/matching/Chopin_Op017No4/Chopin_Op017No4_Levy-1951_pid915406-13.npy"
file_2 = "/home/ijain/ttmp/Chopin_Mazurkas_features/matching/Chopin_Op017No4/Chopin_Op017No4_Afanassiev-2001_pid9130-01.npy"

# Load feature files
F1 = np.load(file_1)
F2 = np.load(file_2)

print(f"F1 shape: {F1.shape}")
print(f"F2 shape: {F2.shape}")

# Compute cost matrix
C = 1 - FlexDTW.L2norm(F1).T @ FlexDTW.L2norm(F2)
print(f"Cost matrix C shape: {C.shape}")

# Set alignment parameters
steps = np.array([[1, 1], [1, 2], [2, 1]])
weights = np.array([1.25, 3.0, 3.0])
beta = 0.1
L = 4000  # Chunk size for Parflex

F1 shape: (12, 9379)
F2 shape: (12, 17642)


Cost matrix C shape: (9379, 17642)


## 2. Path Computation

In [3]:
# Run FlexDTW (same buffer as 04_Align_Benchmarks so path is full-band)
L1, L2 = C.shape[0], C.shape[1]
buffer_flex = min(L1, L2) * (1 - (1 - beta) * min(L1, L2) / max(L1, L2))
print("Running FlexDTW...")
best_cost_flex, wp_flex, D_flex, P_flex, B_flex, debug_flex = FlexDTW.flexdtw(
    C, steps=steps, weights=weights, buffer=buffer_flex
)
print(f"FlexDTW best cost: {best_cost_flex:.6f}")
print(f"FlexDTW path length: {len(wp_flex)}")

# Normalize FlexDTW path to (N, 2) format
if wp_flex.shape[0] == 2:
    wp_flex = wp_flex.T  # Convert (2, N) to (N, 2)
print(f"FlexDTW path shape: {wp_flex.shape}")

# Run Parflex Stage 1
print("\nRunning Parflex Stage 1...")
C_parflex, tiled_result = Parflex.align_system_parflex(
    F1, F2, steps=steps, weights=weights, beta=beta, L=L
)
print(f"Parflex chunks: {tiled_result['n_row']} x {tiled_result['n_col']}")

# Run Parflex Stage 2
print("Running Parflex Stage 2...")
parflex_result = Parflex.parflex_2a(tiled_result, C_parflex, beta=beta, show_fig=False)
wp_parflex = parflex_result['stitched_wp']
print(f"Parflex path length: {len(wp_parflex)}")
print(f"Parflex path shape: {wp_parflex.shape}")

# Ensure Parflex path is (N, 2) format
if wp_parflex.shape[0] == 2:
    wp_parflex = wp_parflex.T
print(f"Parflex normalized path shape: {wp_parflex.shape}")

Running FlexDTW...
FlexDTW best cost: 0.157340
FlexDTW path length: 2
FlexDTW path shape: (9314, 2)

Running Parflex Stage 1...
Parflex chunks: 3 x 5
Running Parflex Stage 2...
[Stage2] transition: chunk(2,1), edge=0, idx=2101 -> chunk(1,1), edge=0, corner_landed=False, S_val=861
[Stage2] transition: chunk(1,1), edge=0, idx=861 -> chunk(1,0), edge=1, corner_landed=False, S_val=-3216
[Stage2] transition: chunk(1,0), edge=1, idx=3216 -> chunk(0,0), edge=0, corner_landed=False, S_val=429
[Stage2] transition: chunk(2,2), edge=0, idx=3900 -> chunk(1,2), edge=0, corner_landed=False, S_val=2520
[Stage2] transition: chunk(1,2), edge=0, idx=2520 -> chunk(1,1), edge=1, corner_landed=False, S_val=-1366
[Stage2] transition: chunk(1,1), edge=1, idx=1366 -> chunk(0,1), edge=0, corner_landed=False, S_val=2100
[Stage2] transition: chunk(0,1), edge=0, idx=2100 -> chunk(0,0), edge=1, corner_landed=False, S_val=-2329
[Stage2] transition: chunk(2,3), edge=0, idx=3321 -> chunk(1,3), edge=0, corner_landed=F

## 3. Intersection Detection

In [4]:
def find_path_intersections(wp1, wp2, tolerance=2):
    """
    Find intersection points between two warping paths.
    
    Parameters:
    -----------
    wp1, wp2 : ndarray, shape (N, 2)
        Warping paths with columns [row, col]
    tolerance : int
        Manhattan distance tolerance for considering points as intersecting
        
    Returns:
    --------
    intersections : list of dict
        Each dict contains:
        - 'point': (row, col) coordinates
        - 'idx1': index in wp1
        - 'idx2': index in wp2
        - 'distance': Manhattan distance (0 if exact match)
    """
    intersections = []
    
    # Convert paths to sets of tuples for exact matches
    wp1_set = set(tuple(p) for p in wp1)
    wp2_set = set(tuple(p) for p in wp2)
    exact_matches = wp1_set & wp2_set
    
    # Find exact matches first
    for point in exact_matches:
        idx1 = np.where((wp1[:, 0] == point[0]) & (wp1[:, 1] == point[1]))[0]
        idx2 = np.where((wp2[:, 0] == point[0]) & (wp2[:, 1] == point[1]))[0]
        if len(idx1) > 0 and len(idx2) > 0:
            intersections.append({
                'point': point,
                'idx1': idx1[0],
                'idx2': idx2[0],
                'distance': 0
            })
    
    # Find near-misses within tolerance
    for i, p1 in enumerate(wp1):
        for j, p2 in enumerate(wp2):
            manhattan_dist = abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])
            if 0 < manhattan_dist <= tolerance:
                # Check if this is not already an exact match
                point_tuple = tuple(p1)
                if point_tuple not in exact_matches:
                    intersections.append({
                        'point': tuple(p1),
                        'idx1': i,
                        'idx2': j,
                        'distance': manhattan_dist
                    })
                    break  # Only add first near-miss for each wp1 point
    
    return intersections

# Find intersections
intersections = find_path_intersections(wp_flex, wp_parflex, tolerance=2)
print(f"Found {len(intersections)} intersection(s)")
for i, inter in enumerate(intersections):
    print(f"  Intersection {i+1}: {inter['point']} (distance: {inter['distance']}, "
          f"flex_idx: {inter['idx1']}, parflex_idx: {inter['idx2']})")

Found 532 intersection(s)
  Intersection 1: (np.int64(6787), np.int64(12249)) (distance: 0, flex_idx: 6765, parflex_idx: 6194)
  Intersection 2: (np.int64(6914), np.int64(12488)) (distance: 0, flex_idx: 6892, parflex_idx: 6321)
  Intersection 3: (np.int64(6486), np.int64(11800)) (distance: 0, flex_idx: 6464, parflex_idx: 5893)
  Intersection 4: (np.int64(6682), np.int64(12102)) (distance: 0, flex_idx: 6660, parflex_idx: 6089)
  Intersection 5: (np.int64(6430), np.int64(11710)) (distance: 0, flex_idx: 6408, parflex_idx: 5837)
  Intersection 6: (np.int64(6467), np.int64(11768)) (distance: 0, flex_idx: 6445, parflex_idx: 5874)
  Intersection 7: (np.int64(6838), np.int64(12349)) (distance: 0, flex_idx: 6816, parflex_idx: 6245)
  Intersection 8: (np.int64(6844), np.int64(12356)) (distance: 0, flex_idx: 6822, parflex_idx: 6251)
  Intersection 9: (np.int64(6707), np.int64(12140)) (distance: 0, flex_idx: 6685, parflex_idx: 6114)
  Intersection 10: (np.int64(6728), np.int64(12172)) (distance: 0

## 4. Cost Matrix Neighborhood Analysis

In [5]:
def extract_cost_neighborhood(C, center_row, center_col, radius=50):
    """
    Extract a submatrix around a center point.
    
    Parameters:
    -----------
    C : ndarray
        Cost matrix
    center_row, center_col : int
        Center coordinates
    radius : int
        Radius of neighborhood
        
    Returns:
    --------
    C_local : ndarray
        Local cost matrix
    row_range : tuple
        (start_row, end_row) in global coordinates
    col_range : tuple
        (start_col, end_col) in global coordinates
    """
    L1, L2 = C.shape
    
    # Calculate bounds
    row_start = max(0, center_row - radius)
    row_end = min(L1, center_row + radius + 1)
    col_start = max(0, center_col - radius)
    col_end = min(L2, center_col + radius + 1)
    
    # Extract submatrix
    C_local = C[row_start:row_end, col_start:col_end].copy()
    
    return C_local, (row_start, row_end), (col_start, col_end)

def map_path_to_local(wp, row_start, row_end, col_start, col_end):
    """
    Map a global warping path to local coordinates.
    
    Parameters:
    -----------
    wp : ndarray, shape (N, 2)
        Global warping path
    row_start, row_end, col_start, col_end : int
        Bounds of local region
        
    Returns:
    --------
    wp_local : ndarray
        Local path (points outside region are filtered out)
    valid_mask : ndarray
        Boolean mask indicating which points are in the local region
    """
    wp = np.asarray(wp)
    valid_mask = ((wp[:, 0] >= row_start) & (wp[:, 0] < row_end) &
                  (wp[:, 1] >= col_start) & (wp[:, 1] < col_end))
    
    wp_local = wp[valid_mask].copy()
    wp_local[:, 0] -= row_start
    wp_local[:, 1] -= col_start
    
    return wp_local, valid_mask

def find_divergence_point(wp1, wp2, intersection_idx1, intersection_idx2):
    """
    Find the first point after intersection where paths diverge.
    
    Parameters:
    -----------
    wp1, wp2 : ndarray, shape (N, 2)
        Warping paths
    intersection_idx1, intersection_idx2 : int
        Indices of intersection point in each path
        
    Returns:
    --------
    divergence_idx1, divergence_idx2 : int
        Indices where paths diverge (or None if they don't diverge)
    divergence_point : tuple
        Coordinates of divergence point
    """
    # Start searching after intersection
    start_idx1 = intersection_idx1 + 1
    start_idx2 = intersection_idx2 + 1
    
    # Find minimum length to check
    min_len = min(len(wp1) - start_idx1, len(wp2) - start_idx2)
    
    for i in range(min_len):
        idx1 = start_idx1 + i
        idx2 = start_idx2 + i
        
        p1 = tuple(wp1[idx1])
        p2 = tuple(wp2[idx2])
        
        if p1 != p2:
            return idx1, idx2, p1
    
    # Paths don't diverge (one ends)
    return None, None, None

## 5. Chunk Boundary Analysis

In [6]:
def identify_chunk_at_point(tiled_result, row, col):
    """
    Identify which chunk contains a given point.
    
    Parameters:
    -----------
    tiled_result : dict
        Result from Parflex.align_system_parflex
    row, col : int
        Global coordinates
        
    Returns:
    --------
    chunk_info : dict or None
        Dictionary with:
        - 'chunk': (i, j) chunk indices
        - 'local_row', 'local_col': local coordinates within chunk
        - 'bounds': (row_start, row_end, col_start, col_end)
        - 'block': block dictionary from tiled_result
    """
    blocks = tiled_result['blocks']
    
    for block in blocks:
        r_start, r_end = block['rows']
        c_start, c_end = block['cols']
        
        if r_start <= row < r_end and c_start <= col < c_end:
            local_row = row - r_start
            local_col = col - c_start
            
            return {
                'chunk': (block['bi'], block['bj']),
                'local_row': local_row,
                'local_col': local_col,
                'bounds': (r_start, r_end, c_start, c_end),
                'block': block
            }
    
    return None

def analyze_chunk_contribution(chunks_dict, chunk_i, chunk_j, intersection_point):
    """
    Analyze how chunk contributes to path cost at intersection point.
    
    Parameters:
    -----------
    chunks_dict : dict
        Dictionary from chunk_flexdtw
    chunk_i, chunk_j : int
        Chunk indices
    intersection_point : tuple
        (row, col) in global coordinates
        
    Returns:
    --------
    analysis : dict
        Analysis results
    """
    chunk_key = (chunk_i, chunk_j)
    if chunk_key not in chunks_dict:
        return None
    
    chunk_data = chunks_dict[chunk_key]
    start_1, end_1, start_2, end_2 = chunk_data['bounds']
    
    # Convert to local coordinates
    local_row = intersection_point[0] - start_1
    local_col = intersection_point[1] - start_2
    
    # Get chunk matrices
    D_chunk = chunk_data['D']
    S_chunk = chunk_data['S']
    B_chunk = chunk_data['B']
    C_chunk = chunk_data.get('C', None)
    
    analysis = {
        'chunk': chunk_key,
        'local_coords': (local_row, local_col),
        'global_coords': intersection_point,
        'bounds': (start_1, end_1, start_2, end_2),
        'D_value': D_chunk[local_row, local_col] if (0 <= local_row < D_chunk.shape[0] and 
                                                      0 <= local_col < D_chunk.shape[1]) else None,
        'S_value': S_chunk[local_row, local_col] if (0 <= local_row < S_chunk.shape[0] and 
                                                      0 <= local_col < S_chunk.shape[1]) else None,
        'B_value': B_chunk[local_row, local_col] if (0 <= local_row < B_chunk.shape[0] and 
                                                      0 <= local_col < B_chunk.shape[1]) else None,
        'C_value': C_chunk[local_row, local_col] if (C_chunk is not None and 
                                                      0 <= local_row < C_chunk.shape[0] and 
                                                      0 <= local_col < C_chunk.shape[1]) else None,
    }
    
    return analysis

## 6. Visualization Functions

In [7]:
def plot_cost_matrix_with_paths(C_local, wp1_local, wp2_local, intersection_point_local, 
                                 divergence_point_local, row_range, col_range, title="Cost Matrix with Paths"):
    """
    Create heatmap of cost matrix with overlaid paths.
    
    Parameters:
    -----------
    C_local : ndarray
        Local cost matrix
    wp1_local, wp2_local : ndarray
        Local warping paths
    intersection_point_local : tuple
        (row, col) in local coordinates
    divergence_point_local : tuple or None
        (row, col) in local coordinates
    row_range, col_range : tuple
        (start, end) ranges for axis labels
    title : str
        Plot title
    """
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Plot cost matrix heatmap
    im = ax.imshow(C_local, cmap='viridis', aspect='auto', origin='lower')
    plt.colorbar(im, ax=ax, label='Cost')
    
    # Plot paths
    if len(wp1_local) > 0:
        ax.plot(wp1_local[:, 1], wp1_local[:, 0], 'b-', linewidth=2, alpha=0.7, 
                label='FlexDTW', marker='o', markersize=3)
    if len(wp2_local) > 0:
        ax.plot(wp2_local[:, 1], wp2_local[:, 0], 'r-', linewidth=2, alpha=0.7, 
                label='Parflex', marker='s', markersize=3)
    
    # Mark intersection point
    if intersection_point_local:
        inter_row, inter_col = intersection_point_local
        ax.plot(inter_col, inter_row, 'go', markersize=15, label='Intersection', 
                markeredgecolor='black', markeredgewidth=2)
        # Add cost annotation
        if 0 <= inter_row < C_local.shape[0] and 0 <= inter_col < C_local.shape[1]:
            cost_val = C_local[inter_row, inter_col]
            ax.annotate(f'C={cost_val:.3f}', 
                       xy=(inter_col, inter_row), 
                       xytext=(10, 10), textcoords='offset points',
                       bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7),
                       fontsize=10)
    
    # Mark divergence point
    if divergence_point_local:
        div_row, div_col = divergence_point_local
        ax.plot(div_col, div_row, 'ro', markersize=15, label='Divergence', 
                markeredgecolor='black', markeredgewidth=2)
        # Add cost annotation
        if 0 <= div_row < C_local.shape[0] and 0 <= div_col < C_local.shape[1]:
            cost_val = C_local[div_row, div_col]
            ax.annotate(f'C={cost_val:.3f}', 
                       xy=(div_col, div_row), 
                       xytext=(10, -20), textcoords='offset points',
                       bbox=dict(boxstyle='round,pad=0.3', facecolor='orange', alpha=0.7),
                       fontsize=10)
    
    ax.set_xlabel('Column (F2 frames)', fontsize=12)
    ax.set_ylabel('Row (F1 frames)', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Set axis labels to global coordinates
    row_start, row_end = row_range
    col_start, col_end = col_range
    
    # Set ticks to show global coordinates
    n_ticks = 5
    row_ticks = np.linspace(0, C_local.shape[0]-1, n_ticks).astype(int)
    col_ticks = np.linspace(0, C_local.shape[1]-1, n_ticks).astype(int)
    
    ax.set_yticks(row_ticks)
    ax.set_yticklabels([row_start + r for r in row_ticks])
    ax.set_xticks(col_ticks)
    ax.set_xticklabels([col_start + c for c in col_ticks])
    
    plt.tight_layout()
    return fig, ax

In [8]:
def plot_path_comparison_zoom(C, wp_flex, wp_parflex, intersection_point, window_size=100):
    """
    Create zoomed-in view around intersection with both paths.
    
    Parameters:
    -----------
    C : ndarray
        Full cost matrix
    wp_flex, wp_parflex : ndarray
        Warping paths
    intersection_point : tuple
        (row, col) global coordinates
    window_size : int
        Size of window around intersection
    """
    inter_row, inter_col = intersection_point
    
    # Extract neighborhood
    C_local, row_range, col_range = extract_cost_neighborhood(
        C, inter_row, inter_col, radius=window_size//2
    )
    
    # Map paths to local
    wp_flex_local, _ = map_path_to_local(wp_flex, *row_range, *col_range)
    wp_parflex_local, _ = map_path_to_local(wp_parflex, *row_range, *col_range)
    
    # Local intersection point
    inter_local = (inter_row - row_range[0], inter_col - col_range[0])
    
    # Find divergence
    intersections = find_path_intersections(wp_flex, wp_parflex, tolerance=2)
    if intersections:
        inter = intersections[0]
        div_idx1, div_idx2, div_point = find_divergence_point(
            wp_flex, wp_parflex, inter['idx1'], inter['idx2']
        )
        if div_point:
            div_local = (div_point[0] - row_range[0], div_point[1] - col_range[0])
        else:
            div_local = None
    else:
        div_local = None
    
    # Plot
    fig, ax = plot_cost_matrix_with_paths(
        C_local, wp_flex_local, wp_parflex_local, inter_local, div_local,
        row_range, col_range, 
        title=f"Zoomed View Around Intersection {intersection_point}"
    )
    
    return fig, ax

In [9]:
def plot_divergence_analysis(C, D_flex, B_flex, wp_flex, wp_parflex, 
                              intersection_idx, divergence_idx, radius=30):
    """
    Visualize divergence point with backpointer analysis.
    
    Parameters:
    -----------
    C, D_flex, B_flex : ndarray
        Cost, accumulated cost, and backpointer matrices
    wp_flex, wp_parflex : ndarray
        Warping paths
    intersection_idx : int
        Index of intersection in wp_flex
    divergence_idx : int
        Index of divergence in wp_flex
    radius : int
        Radius around divergence point
    """
    if divergence_idx is None:
        print("No divergence found")
        return None
    
    div_point = wp_flex[divergence_idx]
    div_row, div_col = div_point
    
    # Extract neighborhood
    C_local, row_range, col_range = extract_cost_neighborhood(
        C, div_row, div_col, radius=radius
    )
    
    # Extract D and B neighborhoods
    D_local = D_flex[row_range[0]:row_range[1], col_range[0]:col_range[1]]
    B_local = B_flex[row_range[0]:row_range[1], col_range[0]:col_range[1]]
    
    # Map paths to local
    wp_flex_local, _ = map_path_to_local(wp_flex, *row_range, *col_range)
    wp_parflex_local, _ = map_path_to_local(wp_parflex, *row_range, *col_range)
    
    # Local divergence point
    div_local = (div_row - row_range[0], div_col - col_range[0])
    
    # Create figure with subplots
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Plot 1: Cost matrix
    im1 = axes[0].imshow(C_local, cmap='viridis', aspect='auto', origin='lower')
    axes[0].plot(wp_flex_local[:, 1], wp_flex_local[:, 0], 'b-', linewidth=2, alpha=0.7, label='FlexDTW')
    axes[0].plot(wp_parflex_local[:, 1], wp_parflex_local[:, 0], 'r-', linewidth=2, alpha=0.7, label='Parflex')
    axes[0].plot(div_local[1], div_local[0], 'ro', markersize=15, markeredgecolor='black', markeredgewidth=2)
    axes[0].set_title('Cost Matrix C', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Column')
    axes[0].set_ylabel('Row')
    axes[0].legend()
    plt.colorbar(im1, ax=axes[0])
    
    # Plot 2: Accumulated cost D
    im2 = axes[1].imshow(D_local, cmap='plasma', aspect='auto', origin='lower')
    axes[1].plot(wp_flex_local[:, 1], wp_flex_local[:, 0], 'b-', linewidth=2, alpha=0.7)
    axes[1].plot(wp_parflex_local[:, 1], wp_parflex_local[:, 0], 'r-', linewidth=2, alpha=0.7)
    axes[1].plot(div_local[1], div_local[0], 'ro', markersize=15, markeredgecolor='black', markeredgewidth=2)
    axes[1].set_title('Accumulated Cost D (FlexDTW)', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Column')
    axes[1].set_ylabel('Row')
    plt.colorbar(im2, ax=axes[1])
    
    # Plot 3: Backpointer visualization
    axes[2].imshow(C_local, cmap='gray', aspect='auto', origin='lower', alpha=0.5)
    
    # Draw arrows for backpointers around divergence point
    steps = np.array([[1, 1], [1, 2], [2, 1]])
    div_row_local, div_col_local = div_local
    
    # Check backpointer at divergence point
    if 0 <= div_row_local < B_local.shape[0] and 0 <= div_col_local < B_local.shape[1]:
        bp_idx = int(B_local[div_row_local, div_col_local])
        if 0 <= bp_idx < len(steps):
            step = steps[bp_idx]
            # Draw arrow showing where FlexDTW came from
            axes[2].arrow(div_col_local, div_row_local, 
                         -step[1], -step[0], 
                         head_width=2, head_length=2, fc='blue', ec='blue', linewidth=3,
                         label='FlexDTW backpointer')
    
    axes[2].plot(wp_flex_local[:, 1], wp_flex_local[:, 0], 'b-', linewidth=2, alpha=0.7, label='FlexDTW')
    axes[2].plot(wp_parflex_local[:, 1], wp_parflex_local[:, 0], 'r-', linewidth=2, alpha=0.7, label='Parflex')
    axes[2].plot(div_local[1], div_local[0], 'ro', markersize=15, markeredgecolor='black', markeredgewidth=2)
    axes[2].set_title('Backpointer Analysis', fontsize=12, fontweight='bold')
    axes[2].set_xlabel('Column')
    axes[2].set_ylabel('Row')
    axes[2].legend()
    
    plt.tight_layout()
    return fig, axes

## 7. Detailed Analysis Report

In [10]:
def generate_divergence_report(C, D_flex, B_flex, S_flex, wp_flex, wp_parflex, 
                                intersection_point, chunks_dict, tiled_result):
    """
    Generate comprehensive text report of divergence analysis.
    
    Parameters:
    -----------
    C, D_flex, B_flex, S_flex : ndarray
        Cost, accumulated cost, backpointer, and start position matrices
    wp_flex, wp_parflex : ndarray
        Warping paths
    intersection_point : tuple
        (row, col) global coordinates
    chunks_dict : dict
        Dictionary from chunk_flexdtw
    tiled_result : dict
        Result from Parflex.align_system_parflex
    """
    inter_row, inter_col = intersection_point
    
    print("=" * 80)
    print("DIVERGENCE ANALYSIS REPORT")
    print("=" * 80)
    print(f"\nIntersection Point: ({inter_row}, {inter_col})")
    
    # Cost matrix values
    print(f"\n--- Cost Matrix Values ---")
    print(f"C[{inter_row}, {inter_col}] = {C[inter_row, inter_col]:.6f}")
    print(f"D_flex[{inter_row}, {inter_col}] = {D_flex[inter_row, inter_col]:.6f}")
    
    # Path indices
    flex_idx = np.where((wp_flex[:, 0] == inter_row) & (wp_flex[:, 1] == inter_col))[0]
    parflex_idx = np.where((wp_parflex[:, 0] == inter_row) & (wp_parflex[:, 1] == inter_col))[0]
    
    if len(flex_idx) > 0 and len(parflex_idx) > 0:
        flex_idx = flex_idx[0]
        parflex_idx = parflex_idx[0]
        
        print(f"\n--- Path Information ---")
        print(f"FlexDTW path index: {flex_idx} / {len(wp_flex)}")
        print(f"Parflex path index: {parflex_idx} / {len(wp_parflex)}")
        
        # Find divergence
        div_idx1, div_idx2, div_point = find_divergence_point(
            wp_flex, wp_parflex, flex_idx, parflex_idx
        )
        
        if div_point:
            div_row, div_col = div_point
            print(f"\n--- Divergence Point ---")
            print(f"Divergence at: ({div_row}, {div_col})")
            print(f"FlexDTW path index: {div_idx1}")
            print(f"Parflex path index: {div_idx2}")
            
            # Cost at divergence
            print(f"\nC[{div_row}, {div_col}] = {C[div_row, div_col]:.6f}")
            print(f"D_flex[{div_row}, {div_col}] = {D_flex[div_row, div_col]:.6f}")
            
            # Backpointer analysis
            bp_idx = int(B_flex[div_row, div_col])
            steps = np.array([[1, 1], [1, 2], [2, 1]])
            if 0 <= bp_idx < len(steps):
                step = steps[bp_idx]
                prev_row = div_row - step[0]
                prev_col = div_col - step[1]
                print(f"\n--- FlexDTW Path Choice ---")
                print(f"Backpointer index: {bp_idx}")
                print(f"Step taken: ({step[0]}, {step[1]})")
                print(f"Previous point: ({prev_row}, {prev_col})")
                print(f"D_flex[{prev_row}, {prev_col}] = {D_flex[prev_row, prev_col]:.6f}")
            
            # Parflex path choice
            if div_idx2 < len(wp_parflex) - 1:
                parflex_next = wp_parflex[div_idx2 + 1]
                parflex_step = (parflex_next[0] - div_row, parflex_next[1] - div_col)
                print(f"\n--- Parflex Path Choice ---")
                print(f"Next point: ({parflex_next[0]}, {parflex_next[1]})")
                print(f"Step taken: ({parflex_step[0]}, {parflex_step[1]})")
        
        # Start position analysis
        print(f"\n--- Start Position (S matrix) ---")
        s_val = S_flex[inter_row, inter_col]
        if s_val > 0:
            start_row, start_col = 0, int(s_val)
            print(f"S[{inter_row}, {inter_col}] = {s_val:.0f} → start at bottom edge: ({start_row}, {start_col})")
        elif s_val < 0:
            start_row, start_col = abs(int(s_val)), 0
            print(f"S[{inter_row}, {inter_col}] = {s_val:.0f} → start at left edge: ({start_row}, {start_col})")
        else:
            print(f"S[{inter_row}, {inter_col}] = 0 → start at origin")
    
    # Chunk analysis
    chunk_info = identify_chunk_at_point(tiled_result, inter_row, inter_col)
    if chunk_info:
        print(f"\n--- Chunk Information ---")
        print(f"Chunk: {chunk_info['chunk']}")
        print(f"Local coordinates: ({chunk_info['local_row']}, {chunk_info['local_col']})")
        print(f"Chunk bounds: rows [{chunk_info['bounds'][0]}, {chunk_info['bounds'][1]}), "
              f"cols [{chunk_info['bounds'][2]}, {chunk_info['bounds'][3]})")
        
        # Analyze chunk contribution
        chunk_analysis = analyze_chunk_contribution(
            chunks_dict, chunk_info['chunk'][0], chunk_info['chunk'][1], intersection_point
        )
        if chunk_analysis:
            print(f"\n--- Chunk-Level Values ---")
            if chunk_analysis['D_value'] is not None:
                print(f"D_chunk[{chunk_analysis['local_coords'][0]}, {chunk_analysis['local_coords'][1]}] = "
                      f"{chunk_analysis['D_value']:.6f}")
            if chunk_analysis['S_value'] is not None:
                print(f"S_chunk[{chunk_analysis['local_coords'][0]}, {chunk_analysis['local_coords'][1]}] = "
                      f"{chunk_analysis['S_value']:.0f}")
            if chunk_analysis['C_value'] is not None:
                print(f"C_chunk[{chunk_analysis['local_coords'][0]}, {chunk_analysis['local_coords'][1]}] = "
                      f"{chunk_analysis['C_value']:.6f}")
            
            # Check if on chunk boundary
            block = chunk_info['block']
            rows, cols = block['Ck_shape']
            local_r, local_c = chunk_info['local_row'], chunk_info['local_col']
            
            on_top_edge = (local_r == rows - 1)
            on_right_edge = (local_c == cols - 1)
            
            if on_top_edge or on_right_edge:
                print(f"\n--- Chunk Boundary Analysis ---")
                if on_top_edge:
                    print(f"Point is on TOP edge of chunk")
                if on_right_edge:
                    print(f"Point is on RIGHT edge of chunk")
                
                # Check previous chunk costs
                chunk_i, chunk_j = chunk_info['chunk']
                if chunk_i > 0 or chunk_j > 0:
                    print(f"\nThis path may incorporate costs from previous chunks.")
    
    print("\n" + "=" * 80)

## 8. Main Execution

In [11]:
# Create output directory
output_dir = Path("path_divergence_analysis")
output_dir.mkdir(exist_ok=True)

# Process each intersection
if len(intersections) > 0:
    # Use first intersection for detailed analysis
    inter = intersections[0]
    intersection_point = inter['point']
    
    print(f"\nAnalyzing intersection at {intersection_point}")
    print(f"FlexDTW index: {inter['idx1']}, Parflex index: {inter['idx2']}")
    
    # Generate detailed report
    generate_divergence_report(
        C, D_flex, B_flex, P_flex, wp_flex, wp_parflex,
        intersection_point, tiled_result['chunks_dict'], tiled_result
    )
    
    # Create visualizations
    print("\nGenerating visualizations...")
    
    # 1. Zoomed view around intersection
    fig1, ax1 = plot_path_comparison_zoom(C, wp_flex, wp_parflex, intersection_point, window_size=200)
    fig1.savefig(output_dir / "cost_matrix_intersection.png", dpi=150, bbox_inches='tight')
    print(f"Saved: {output_dir / 'cost_matrix_intersection.png'}")
    
    # 2. Divergence analysis
    div_idx1, div_idx2, div_point = find_divergence_point(
        wp_flex, wp_parflex, inter['idx1'], inter['idx2']
    )
    if div_point:
        fig2, axes2 = plot_divergence_analysis(
            C, D_flex, B_flex, wp_flex, wp_parflex,
            inter['idx1'], div_idx1, radius=30
        )
        if fig2:
            fig2.savefig(output_dir / "divergence_analysis.png", dpi=150, bbox_inches='tight')
            print(f"Saved: {output_dir / 'divergence_analysis.png'}")
    
    # 3. Detailed neighborhood view
    C_local, row_range, col_range = extract_cost_neighborhood(
        C, intersection_point[0], intersection_point[1], radius=50
    )
    wp_flex_local, _ = map_path_to_local(wp_flex, *row_range, *col_range)
    wp_parflex_local, _ = map_path_to_local(wp_parflex, *row_range, *col_range)
    inter_local = (intersection_point[0] - row_range[0], intersection_point[1] - col_range[0])
    
    if div_point:
        div_local = (div_point[0] - row_range[0], div_point[1] - col_range[0])
    else:
        div_local = None
    
    fig3, ax3 = plot_cost_matrix_with_paths(
        C_local, wp_flex_local, wp_parflex_local, inter_local, div_local,
        row_range, col_range, title="Detailed Neighborhood View"
    )
    fig3.savefig(output_dir / "divergence_zoom.png", dpi=150, bbox_inches='tight')
    print(f"Saved: {output_dir / 'divergence_zoom.png'}")
    
    plt.show()
    
else:
    print("No intersections found between paths.")

No intersections found between paths.
